## Scrap que busca arquivos no site Dados Abertos da PRF
https://www.gov.br/prf/pt-br/acesso-a-informacao/dados-abertos/dados-abertos-acidentes
- Dado o ano e planilhas a função faz o download dos aquivos e faz o upload na Google Cloud Storage
- Necessário fazer as devidas autorizações para acesso à Google Cloud Storage


### Arquivo para ser rodado no DataProc
##### Cluster deve ser hospedado em São Paulo {site da PRF não é acessível de outros lugares}
##### Não é necessário importar módulo da GCS as funções estão nesse noteboo

In [17]:
# Importa biblioteca
from google.cloud import storage
import wget
import io, os
#from GCS import *
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.error import *
import sys
from google.cloud import storage
storage_client = storage.Client()

### Abre site e faz leitura do html

In [18]:
# Abre o link, retira HTML e filtra somente a tag HTML desejada 
# as tags de interesse são <a href class='external-link' para o ano e link do arquivo
# e h2 para o tipo de arquivo Ocorrências ou Pessoas
try:
    html=urlopen("https://www.gov.br/prf/pt-br/acesso-a-informacao/dados-abertos/dados-abertos-acidentes")
    bs=BeautifulSoup(html.read(), 'html.parser')
    links=bs.findAll('a', {'class':'external-link'})
    listagrupos=bs.findAll('h2')
except (URLError, HTTPError) as e:
    print('Não foi possivel ler o site', e)


In [19]:
#Remove tag h2 não necessária
listagrupos.pop(0)

<h2></h2>

In [20]:
# Cria dicionário arquivos que contem o tipo da tabela e um dicionario com ano e link
# Exemplo: {'Agrupados por ocorrência':{'ano': http://link_para_o_download}}
n=0 #controle
arquivos={} #dicionário armazena os links
z=0 #controle
for i in links:
    ano=i.get_text()
    if(ano[:4]=='2022'):
        planilha=listagrupos[n].get_text()
        arquivos[planilha]={}
        n=n+1
        z=1
    if(z==1 and ano!=''):
        link=i['href']
        arquivos[planilha].update({ano[:4]:link})


In [21]:
def cria_bucket(bucket_nome):
    """
    Cria bucket com localização pré-definida
    """
    # bucket_nome = "nome-da-nova-bucket"

    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_nome)
    bucket.storage_class = "COLDLINE"
    new_bucket = storage_client.create_bucket(bucket, location="us")

    print(
        "Bucket {} criada em {} com classe {}".format(
            new_bucket.name, new_bucket.location, new_bucket.storage_class
        )
    )
    return new_bucket

In [22]:
def upload_objeto(bucket_name, source_file_name, destination_blob_name):
    """Upload de aquivos na bucket"""
    # O ID da GCS bucket
    # bucket_name = "nome-da-bucket"
    # O caminho do arquivo a ser enviado
    # source_file_name = "local/caminho/para/arquivo"
    # O ID do objeto na bucket
    # destination_blob_name = "nome-objeto-bucket"

    
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name, timeout=(500000,5000000.0))

    print(
        "Arquivo {} enviado para {}.".format(
            source_file_name, destination_blob_name
        )
    )

In [23]:
def upload(bucket,ano,planilha, caminho):
    '''Função que faz o download dos arquivos .zip e upload para a bucket'''
    # bucket = "nome-da-nova-bucket"
    # ano = a partir de que ano será o download
    # planilha = qual planilha: Pessoas ou Ocorrências
    # caminho = define em quais pastas será salvo na GCS
    
    for i,j in arquivos.items():
        if(i==planilha):
            for k, v in j.items():
                if(int(k)>=ano and int(k)<2022):
                    if(int(k)==2021):
                        v=v+'/download'
                    arquivoslocal= wget.download(v)
                    upload_objeto(bucket,arquivoslocal,caminho+arquivoslocal)
                    os.remove(arquivoslocal)

In [24]:
#Define a pasta dadosbrutos para envio dos arquivos
# define a bucket para envio
#cria_objeto é função do módulo GCS.py
caminho="dadosbrutos/pessoas/"
bucket='teste_jasdhfkajsdhaf'
cria_bucket(bucket)
upload(bucket,2017,'Agrupados por pessoa', caminho)


Bucket teste_jasdhfkajsdhaf criada em US com classe COLDLINE
Arquivo acidentes2021.zip enviado para dadosbrutos/pessoas/acidentes2021.zip.
Arquivo acidentes2020.zip enviado para dadosbrutos/pessoas/acidentes2020.zip.
Arquivo acidentes2019.zip enviado para dadosbrutos/pessoas/acidentes2019.zip.
Arquivo acidentes2018.zip enviado para dadosbrutos/pessoas/acidentes2018.zip.
Arquivo acidentes2017.rar enviado para dadosbrutos/pessoas/acidentes2017.rar.


In [25]:
caminho='dadosbrutos/ocorrencias/'
upload(bucket,2017, 'Agrupados por ocorrência', caminho)

Arquivo datatran2021.zip enviado para dadosbrutos/ocorrencias/datatran2021.zip.
Arquivo datatran2020.zip enviado para dadosbrutos/ocorrencias/datatran2020.zip.
Arquivo datatran2019.zip enviado para dadosbrutos/ocorrencias/datatran2019.zip.
Arquivo datatran2018.zip enviado para dadosbrutos/ocorrencias/datatran2018.zip.
Arquivo datatran2017.zip enviado para dadosbrutos/ocorrencias/datatran2017.zip.
